In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
np.random.seed(123467)
cuts=np.sort(np.random.randint(0,140,19))
cuts=np.append([0],np.append(cuts,[140]))
cuts = cuts[1:]-cuts[:-1]
df_1 = pd.DataFrame(np.cumprod(1-np.repeat(np.random.randint(-5,6,20), cuts)/10), columns=['Level'])
df_2 = pd.DataFrame({'year':2011,
              'month': 6,
              'day':[21]*12+[22]*6,
              'hour':list(range(0,24,2))+list(range(0,11,2)),
              'threshold':np.random.randint(0,90,size=18)/100})

# Assignment 7.1
The DataFrame `df_1` contains measurements of power levels taken at fifteen minute intervals. The measurements started on Jun 21, 2011 at midnight. Create a datetime index for the DataFrame, labeling each observation with its observation time.
<details><summary>Hint 1</summary>
    <p>
        Pandas has the function `data_range`, that can generate a DataTimeIndex
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        A new index can be assigned to a DataFrame by assigning it to the `index` attribute of the DataFrame
    </p>
</details>

In [ ]:
df_1.index = pd.date_range('21/6/2011', periods=df_1.shape[0], freq='15Min')

# Assignment 7.2
The DataFrame `df_2` contains the policy date for the threshold of the power level. The dates and times for the threshold is spread over the columns `year`, `month`, `day` and `hour`. Create a pandas Series that has the datetime stamp as index and the threshold as values
<details><summary>Hint 1</summary>
    <p>
        The pandas function `to_datetime` can transform appropriately named columns in a DataFrame to a DateTimeIndex
    </p>
</details>

In [ ]:
df_2.index = pd.to_datetime(df_2.drop('threshold', axis=1))
df_2 = df_2['threshold']

# Assignment 7.3
The datetime stamp of the threshold indicates the start time of the threshold. We wish to investigate when the level in `df_1` rises above the threshold in `df_2`. Convert `df_2` to have records at 15 minute interval, filling in the current value of the threshold for the newly created records.
<details><summary>Hint 1</summary>
    <p>
        The frequency of a DataFrame with a DateTimeIndex can be changed with the method `asfreq`
    </p>
</details>
<details><summary>Hint 1</summary>
    <p>
        Fill methods `ffill` and `pad` both fill missing values with the last known value
    </p>
</details>

In [ ]:
df_2 = df_2.asfreq('15Min', method='ffill')

# Assignment 7.4
Calculate how many minute the level in `df_1` was above the set threshold in `df_2` for the observations between 22:00 and 6:00.
<details><summary>Hint 1</summary>
    <p>
        The method `between_time` makes it possible to set a start and an end time for filtering a DataFrame. A filter set on a time interval is repeated for every date in the DateTimeIndex
    </p>
</details>

In [ ]:
df_1.merge(df_2.to_frame(), right_index=True, left_index=True).assign(above=lambda r: r['Level']>r['threshold'])['above'].between_time(start_time='22:00', end_time='06:00').sum()*15

# Assignment 7.5
Find the length of the longest period, where the rate of change for the Level in `df_1` is growing. That is the longest period where the second derivative
$$
\frac{\Delta^2f}{\Delta t^2} = \frac{\frac{f(t_n)-f(t_{n-1})}{t_n-t_{n-1}}-\frac{f(t_{n-1})-f(t_{n-2})}{t_{n-1}-t_{n-2}}}{t_n-t_{n-1}}
$$
is greater than zero.
<details><summary>Hint 1</summary>
    <p>
        Both DataFrames and Series have the method `rolling`, that allows you to apply a function to a rolling subset of the rows.
    </p>
</details>
<details><summary>Hint 2</summary>
    <p>
        A cummulative sum over a boolean Series can be used to group records together. All `False` values will be in the same group as the last `True` before them.
    </p>
</details>

In [ ]:
def deriv(s):
    return (s.iloc[1]-s.iloc[0])/(s.index[1]-s.index[0]).seconds

def second_deriv(s):
    return (deriv(s.iloc[1:3])-deriv(s.iloc[:2]))/(s.index[2]-s.index[1]).seconds

gt_zero = df_1['Level'].rolling(3, min_periods=3).apply(second_deriv, raw=False)>0

gt_zero[gt_zero].groupby((~gt_zero).cumsum()[gt_zero]).size().max()